<a href="https://colab.research.google.com/github/GoldPapaya/info256-applied-nlp/blob/main/10.llms/Shot%20Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/10.llms/Shot%20Selection.ipynb)

Given a dataset of labeled (x,y) examples, how do you select the examples used as shots in few-shot learning?  [Albalak et al. 2024](https://arxiv.org/pdf/2402.16827) provides a great overview of such data selection strategies for large language models, but in this homework we'll focus on a simple one from [Liu et al. 2021](https://arxiv.org/pdf/2101.06804): given a test data point $x_{test}$, find the $n$ closest points in the training data (by cosine similarity to sentence embeddings of those points).  In this framework, each data point that you want to predict will have a different set of shots selected for it.

This is the algorithm we will be implementing:

<img src="attachment:2d5c41aa-ab93-49a7-bd79-84cd5fb8967f.png" style="max-width: 400px;"/>

## Dataset

Here's the data we'll use.  This is the subject classification task from the Library of Congress ($x$ = the title of a book and $y$ = a choice of a single subject classification). Be sure to adjust the prompt categories in the example below to reflect the classes you're trying to predict.

In [1]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/loc/train.200.tsv
!wget https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/loc/dev.100.tsv

--2025-10-23 23:45:47--  https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/loc/train.200.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17746 (17K) [text/plain]
Saving to: ‘train.200.tsv’

train.200.tsv       100%[===================>]  17.33K  --.-KB/s    in 0s      

2025-10-23 23:45:47 (187 MB/s) - ‘train.200.tsv’ saved [17746/17746]

--2025-10-23 23:45:47--  https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/loc/dev.100.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9495 (9.3K) 

In [2]:
lc_classification = {
    "A": "GENERAL WORKS",
    "B": "PHILOSOPHY. PSYCHOLOGY. RELIGION",
    "C": "AUXILIARY SCIENCES OF HISTORY",
    "D": "WORLD HISTORY AND HISTORY OF EUROPE, ASIA, AFRICA, AUSTRALIA, NEW ZEALAND, ETC.",
    "E": "HISTORY OF THE AMERICAS",
    "F": "HISTORY OF THE AMERICAS",
    "G": "GEOGRAPHY. ANTHROPOLOGY. RECREATION",
    "H": "SOCIAL SCIENCES",
    "J": "POLITICAL SCIENCE",
    "K": "LAW",
    "L": "EDUCATION",
    "M": "MUSIC AND BOOKS ON MUSIC",
    "N": "FINE ARTS",
    "P": "LANGUAGE AND LITERATURE",
    "Q": "SCIENCE",
    "R": "MEDICINE",
    "S": "AGRICULTURE",
    "T": "TECHNOLOGY",
    "U": "MILITARY SCIENCE",
    "V": "NAVAL SCIENCE",
    "Z": "BIBLIOGRAPHY. LIBRARY SCIENCE. INFORMATION RESOURCES (GENERAL)"
}

def read_data(filepath):
    labels = []
    texts = []
    with open(filepath, "r") as f:
        for line in f:
            label, text, _ = line.strip().split("\t")
            labels.append(lc_classification[label])
            texts.append(text)
    return texts, labels

In [3]:
trainX, trainY = read_data("./train.200.tsv")

In [4]:
devX, devY = read_data("./dev.100.tsv")

## Nearest neighbors

We will first embed all of the training examples, then define a nearest neighbors function.

In [5]:
import torch

In [6]:
from sentence_transformers import SentenceTransformer

def create_nn_search(sentences):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(sentences)

    def search(query, k=5):
        # return the top k nearest indices
        query_vec = model.encode(query)
        similarities = model.similarity(query_vec, embeddings)
        topk = torch.argsort(similarities.squeeze())[-k:]
        return list(topk)[::-1]
    return search



In [7]:
nn_search = create_nn_search(trainX)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

What are the three nearest neighbors in the training set for the title "Flemish paintings in the Metropolitan Museum of Art "?

## LLM Prompting

We will evaluate 3-shot prompting with a random sample as well as with the 3 nearest neighbors to the target.

In [13]:
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

In [14]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B", device_map="cuda", dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [15]:
from textwrap import dedent
import itertools

def classify_with_prompt(labels, shots, target_x, thinking=False):
    newline = "\n"
    system_prompt = dedent(f"""
        You're a helpful assistant for text classification. You'll be given an input text and need to output a single choice from the following set of categories:
        {newline.join(labels)}
        Pick one of those labels and do not generate any other text.
    """).strip()

    messages = [
        {"role": "system", "content": system_prompt},
    ]
    messages += itertools.chain(*[
        ({"role": "user", "content": shot["X"]}, {"role": "assistant", "content": shot["y"]})
        for shot in shots
    ])
    messages += [
        {"role": "user", "content": target_x}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=thinking # Switches between thinking and non-thinking modes. Default is True.
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated = model.generate(
        **model_inputs,
        max_new_tokens=100
    )

    # let's break this down:
    #                      | we take the element of the batch (our batch size is 1)
    #                      |  |-----------------------------| skip our original input
    output_ids = generated[0][len(model_inputs.input_ids[0]):].tolist()

    # decode into token space
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")

In [16]:
import random

def predict_random_sample(trainX, trainY, devX, k=3):
    predicted = []
    for target_x in tqdm(devX):
        inds = random.sample(range(len(trainX)), k)
        shots = [{"X": trainX[i], "y": trainY[i]} for i in inds]
        labels = lc_classification.values()
        prediction = classify_with_prompt(labels, shots, target_x)
        predicted.append(prediction)
    return predicted

def predict_nn_sample(trainX, trainY, devX, k=3):
    predicted = []
    # search_fn(query_sentence, k) returns the top k indices
    # of sentences closest to the query_sentence
    search_fn = create_nn_search(trainX)
    for target_x in tqdm(devX):
        # IMPLEMENT ME
    return predicted

IndentationError: expected an indented block after 'for' statement on line 18 (ipython-input-2879183576.py, line 20)

In [ ]:
rand_1_sample_acc = sum([pred == target for pred, target in zip(predict_random_sample(trainX, trainY, devX, k=1), devY)]) / len(devY)
nn_1_sample_acc = sum([pred == target for pred, target in zip(predict_nn_sample(trainX, trainY, devX, k=1), devY)]) / len(devY)

rand_3_sample_acc = sum([pred == target for pred, target in zip(predict_random_sample(trainX, trainY, devX), devY)]) / len(devY)
nn_3_sample_acc = sum([pred == target for pred, target in zip(predict_nn_sample(trainX, trainY, devX), devY)]) / len(devY)

print(f"""
Random 1-shot sample acc: {rand_1_sample_acc}
Nearest neighbor 1-shot sample acc: {nn_1_sample_acc}
---
Random 3-shot sample acc: {rand_3_sample_acc}
Nearest neighbor 3-shot sample acc: {nn_3_sample_acc}
""")